In [1]:
import numpy as np
import pandas as pd
import os

from flipside import Flipside
flipside = Flipside("023b83a2-e411-4849-a961-3742c87bc9c9", "https://api-v2.flipsidecrypto.xyz")

We first set the name of the liquidity pool we want to query data from.

In [2]:
pool_name = "USDC-WETH 500 10"

This write the sql code and start query.

In [3]:
sql = """
SELECT pool_name, action, block_number, block_timestamp, 
       liquidity_provider, 
       amount0_adjusted, token0_symbol, amount0_usd,
       amount1_adjusted, token1_symbol, amount1_usd,
       tick_lower, tick_upper
FROM ethereum.uniswapv3.ez_lp_actions
""" +\
" WHERE pool_name = '" + pool_name + "' " +\
"""
ORDER BY block_number DESC
"""

# Since there is a 30mb rate limit for each page, we need to split the data in seperate pages, we will merge them later
# If we are retrieving a dataset with more columns, we might want to set page_size to a lower number
query_result_set = flipside.query(sql, page_number = 1, page_size = 30000)

Here, we run a loop to merge the pages into one full dataset.

In [4]:
current_page_number = 1
page_size = 30000
# We first set total_pages to 2 (any number larger than current_page_number will do) for the loop to run, we will retrieve the actual total pages in the loop
total_pages = 2

# All the data will be stored in a list called "all_rows"
all_rows = []

while current_page_number <= total_pages:
  results = flipside.get_query_results(
    query_result_set.query_id,
    page_number=current_page_number,
    page_size=page_size
  )

  total_pages = results.page.totalPages
  if results.records:
      all_rows = all_rows + results.records
  
  current_page_number += 1

# Merge all rows to a dataframe
df = pd.DataFrame(all_rows)
df.drop(columns = ['__row_index'], inplace = True)
df.head(10)

,pool_name,action,block_number,block_timestamp,liquidity_provider,amount0_adjusted,token0_symbol,amount0_usd,amount1_adjusted,token1_symbol,amount1_usd,tick_lower,tick_upper
0,USDC-WETH 500 10,INCREASE_LIQUIDITY,17614897,2023-07-03T17:23:47.000Z,0x6ceddd716ac124fde6e3a123c533e62e201690c5,1656.090040,USDC,NaN,0.949999,WETH,NaN,184210,214170
1,USDC-WETH 500 10,DECREASE_LIQUIDITY,17614866,2023-07-03T17:17:35.000Z,0x9c3c6cf9d29ab9e9e14503dbfc9ad8bb2a0e37ef,201987.193521,USDC,NaN,0.000000,WETH,NaN,200540,201120
2,USDC-WETH 500 10,DECREASE_LIQUIDITY,17614850,2023-07-03T17:14:23.000Z,0xe31aec7168259c8f7ccdad01acade55d59977624,34259.333850,USDC,NaN,0.000000,WETH,NaN,200690,201230
3,USDC-WETH 500 10,INCREASE_LIQUIDITY,17614835,2023-07-03T17:11:23.000Z,0x92b3ca32614065006582e3a675e30b944785d10d,10953.103634,USDC,NaN,2.316687,WETH,NaN,200340,200930
4,USDC-WETH 500 10,DECREASE_LIQUIDITY,17614827,2023-07-03T17:09:35.000Z,0x6046945c5b5ef5933b8e73a98a6ad7bf3e031df7,105889.614633,USDC,NaN,8525.892186,WETH,NaN,200500,200510
5,USDC-WETH 500 10,INCREASE_LIQUIDITY,17614827,2023-07-03T17:09:35.000Z,0x6046945c5b5ef5933b8e73a98a6ad7bf3e031df7,174381.523001,USDC,NaN,8490.958941,WETH,NaN,200500,200510
6,USDC-WETH 500 10,DECREASE_LIQUIDITY,17614824,2023-07-03T17:08:59.000Z,0xeca2e2d894d19778939bd4dfc34d2a3c45e96456,174419.081107,USDC,NaN,8492.787714,WETH,NaN,200500,200510
7,USDC-WETH 500 10,INCREASE_LIQUIDITY,17614824,2023-07-03T17:08:59.000Z,0xeca2e2d894d19778939bd4dfc34d2a3c45e96456,276233.106754,USDC,NaN,8440.859298,WETH,NaN,200500,200510
8,USDC-WETH 500 10,DECREASE_LIQUIDITY,17614803,2023-07-03T17:04:47.000Z,0xfb6a24b3fab108745c3bcef78b5b566b21be65bf,699.999999,USDC,NaN,0.000000,WETH,NaN,200820,201940
9,USDC-WETH 500 10,INCREASE_LIQUIDITY,17614695,2023-07-03T16:43:11.000Z,0xce7a7f3991f6f11601d428a8772a7f9f1892be76,10291.462850,USDC,NaN,0.000000,WETH,NaN,200520,200560


Lastly, we want to save the data in our desire repository.

In [5]:
df.to_csv("/Users/zhicong/Dropbox/DEXs/uniswapv3_lp_actions/" + pool_name + ".csv", index=False)